<a href="https://colab.research.google.com/github/Mubarok982/forecasting/blob/main/Forecasting_wether.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
import pandas as pd
import tensorflow as tf

In [2]:
data = pd.read_csv('https://drive.google.com/uc?id=15RfMD9lNkpS3cVN7j3_dsJKZ8_5RJG5z')
data.head()

,date,meantemp,humidity,wind_speed,meanpressure
0,2013-01-01,10.000000,84.500000,0.000000,1015.666667
1,2013-01-02,7.400000,92.000000,2.980000,1017.800000
2,2013-01-03,7.166667,87.000000,4.633333,1018.666667
3,2013-01-04,8.666667,71.333333,1.233333,1017.166667
4,2013-01-05,6.000000,86.833333,3.700000,1016.500000


In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1462 entries, 0 to 1461
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   date          1462 non-null   object 
 1   meantemp      1462 non-null   float64
 2   humidity      1462 non-null   float64
 3   wind_speed    1462 non-null   float64
 4   meanpressure  1462 non-null   float64
dtypes: float64(4), object(1)
memory usage: 57.2+ KB


## **sliding windows**

In [7]:
def windowed_dataset(series, window_size, batch_size, shuffle_buffer):
  series = tf.expand_dims(series, axis=-1)
  ds = tf.data.Dataset.from_tensor_slices(series)
  ds = ds.window(window_size + 1, shift=1, drop_remainder=True)
  ds = ds.flat_map(lambda w: w.batch(window_size + 1))
  ds = ds.shuffle(shuffle_buffer)
  ds = ds.map(lambda w: (w[:-1], w[-1]))
  return ds.batch(batch_size).prefetch(1)

In [10]:
dates = data['date'].values
temp = data['meantemp'].values

In [12]:
train_set = windowed_dataset(temp, window_size=60, batch_size=100, shuffle_buffer=1000)

In [13]:
for data in train_set:
  print(data)
  break

(<tf.Tensor: shape=(100, 60, 1), dtype=float64, numpy=
array([[[18.5       ],
        [17.875     ],
        [18.25      ],
        ...,
        [14.875     ],
        [12.        ],
        [12.28571429]],

       [[33.25      ],
        [34.625     ],
        [31.5       ],
        ...,
        [27.25      ],
        [29.875     ],
        [30.875     ]],

       [[32.125     ],
        [29.2       ],
        [30.28571429],
        ...,
        [34.14285714],
        [29.83333333],
        [30.14285714]],

       ...,

       [[16.5       ],
        [16.375     ],
        [18.125     ],
        ...,
        [28.        ],
        [29.375     ],
        [29.125     ]],

       [[29.25      ],
        [28.625     ],
        [27.875     ],
        ...,
        [24.75      ],
        [23.875     ],
        [23.375     ]],

       [[ 9.5       ],
        [14.        ],
        [13.83333333],
        ...,
        [29.16666667],
        [23.83333333],
        [25.25      ]]])>, <tf.Tensor: 

In [18]:
(<tf.Tensor: shape=(3, 2, 1), dtype=float64, numpy=
array([[[31.25 ],
        [31.   ]],

       [[30.25 ],
        [29.875]],

       [[13.75 ],
        [13.375]]])>, <tf.Tensor: shape=(3, 1, 1), dtype=float64, numpy=
array([[[28.75]],

       [[30.75]],

       [[13.25]]])>)

SyntaxError: invalid syntax (ipython-input-18-1439504900.py, line 1)

In [17]:
model = tf.keras.models.Sequential([
  tf.keras.layers.LSTM(60, return_sequences=True),
  tf.keras.layers.LSTM(60),
  tf.keras.layers.Dense(30, activation="relu"),
  tf.keras.layers.Dense(10, activation="relu"),
  tf.keras.layers.Dense(1),
])

optimizer = tf.keras.optimizers.SGD(learning_rate=1e-5, momentum=0.9)
model.compile(loss=tf.keras.losses.Huber(),
              optimizer=optimizer,
              metrics=["mae"])